In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold
from keras.layers import Input, Dense, Conv1D, Flatten, MaxPooling1D, Conv2D, MaxPooling2D, AveragePooling2D, Dropout, Reshape, normalization
from keras.models import Model
from keras.utils import to_categorical
import keras.backend as K
from keras.layers.recurrent import LSTM
from sklearn import metrics
import random
from keras.regularizers import l1, l2

Using TensorFlow backend.


In [32]:
def precision(y_true, y_pred):
    # Calculates the precision
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def recall(y_true, y_pred):
    # Calculates the recall
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def f1(test_Y, pre_test_y):
    #calculate the F1-score
    Precision = precision(test_Y, pre_test_y)
    Recall = recall(test_Y, pre_test_y)
    f1 = 2 * ((Precision * Recall) / (Precision + Recall + K.epsilon()))
    return f1 

def TP(test_Y,pre_test_y):
    #calculate numbers of true positive samples
    TP = K.sum(K.round(K.clip(test_Y * pre_test_y, 0, 1)))#TP
    return TP

def FN(test_Y,pre_test_y):
     #calculate numbers of false negative samples
    TP = K.sum(K.round(K.clip(test_Y * pre_test_y, 0, 1)))#TP
    P=K.sum(K.round(K.clip(test_Y, 0, 1)))
    FN = P-TP #FN=P-TP
    return FN

def TN(test_Y,pre_test_y):
    #calculate numbers of True negative samples
    TN=K.sum(K.round(K.clip((test_Y-K.ones_like(test_Y))*(pre_test_y-K.ones_like(pre_test_y)), 0, 1)))#TN
    return TN

def FP(test_Y,pre_test_y):
    #calculate numbers of False positive samples
    N = (-1)*K.sum(K.round(K.clip(test_Y-K.ones_like(test_Y), -1, 0)))#N
    TN=K.sum(K.round(K.clip((test_Y-K.ones_like(test_Y))*(pre_test_y-K.ones_like(pre_test_y)), 0, 1)))#TN
    FP=N-TN
    return FP

def dnn_model(train_X, train_Y, test_X, test_Y, lr, epoch, batch_size):
    train_X = np.expand_dims(train_X, 2)
    test_X = np.expand_dims(test_X, 2)
    inputs = Input(shape = (train_X.shape[1], train_X.shape[2]))
    x = Conv1D(32, kernel_size = 3, strides = 1, padding = 'valid', activation = 'relu')(inputs)
    x = MaxPooling1D(pool_size = 2, strides = 2, padding = 'same')(x)
    x = Flatten()(x)
    x = Dropout(0.5)(x)
    x = Dense(32, activation = 'relu',kernel_regularizer = l2(1e-5))(x)
    x = Dense(16, activation = 'relu',kernel_regularizer = l2(1e-5))(x)
    x = Dense(8, activation = 'relu',kernel_regularizer = l2(1e-5))(x)
    predictions = Dense(1, activation = 'sigmoid')(x)
    model = Model(inputs = inputs, outputs = predictions)
    print("model")
    model.compile(optimizer = 'RMSProp',
                  loss = 'mean_squared_error',
                  metrics = ['acc',precision,recall,f1,TP,FN,TN,FP])
    print("compile")
    model.fit(train_X, train_Y, epochs = epoch, batch_size = 32, validation_data = (test_X, test_Y), shuffle = True)
    model.save('6fea_idx12Scale_infoGainSorted50.h5') #save model
    pre_test_y = model.predict(test_X, batch_size = 50)
    pre_train_y = model.predict(train_X, batch_size = 50)
    test_auc = metrics.roc_auc_score(test_Y, pre_test_y)
    train_auc = metrics.roc_auc_score(train_Y, pre_train_y)
    print("train_auc: ", train_auc)
    print("test_auc: ", test_auc) 
    return test_auc


In [33]:
# split data and output result
data = np.array(pd.read_csv("6fea_idx12Scale_infoGainSorted50.csv"))#inputfile
X1 = data[0:6975, 1:]#6975 is the number of positive samples in training set, '1' is the label of positive sample
Y1 = data[0:6975, 0]#'0' is the label of negative sample
X2 = data[6975:, 1:]
Y2 = data[6975:, 0]
X = np.concatenate([X1, X2], 0)
Y = np.concatenate([Y1, Y2], 0)
#Y = Y.reshape((Y.shape[0], -1))
print (X)
print ("X.shape: ", X.shape)
print ("Y.shape: ", Y.shape)

lr = 0.2 #learning rate
epoch = 10
batch_size = 32
kf = KFold(n_splits = 10, shuffle = True, random_state = 42)
#kf = KFold(n_splits = 5, shuffle = False)
kf = kf.split(X)

test_aucs = []
for i, (train_fold, validate_fold) in enumerate(kf):
    print("\n\ni: ", i)
    test_auc = dnn_model(X[train_fold], Y[train_fold], X[validate_fold], Y[validate_fold], lr, epoch, batch_size)
    test_aucs.append(test_auc)
w = open("6fea_idx12Scale_infoGainSorted50Result.txt", "w")
for j in test_aucs: 
    w.write(str(j) + ',')
w.write('\n')
w.write(str(np.mean(test_aucs)) + '\n')
w.close()


[[0.315789  0.428571  0.623001  ... 0.620922  0.166667  0.       ]
 [0.315789  0.428571  0.627306  ... 0.580021  0.166667  0.       ]
 [0.421053  0.428571  0.311193  ... 0.486997  0.166667  0.       ]
 ...
 [0.578947  0.190476  0.777983  ... 0.729916  0.        0.166667 ]
 [0.368421  0.238095  0.623001  ... 0.482584  0.25      0.0833333]
 [0.315789  0.047619  0.623001  ... 0.132024  0.0833333 0.0833333]]
X.shape:  (13950, 48)
Y.shape:  (13950,)


i:  0
model
compile
Train on 12555 samples, validate on 1395 samples
Epoch 1/10
12555/12555 [==============================] - 1s 119us/step - loss: 0.2074 - acc: 0.6761 - precision: 0.6468 - recall: 0.7643 - f1: 0.6829 - TP: 12.2595 - FN: 3.7583 - TN: 9.3410 - FP: 6.5878 - val_loss: 0.1905 - val_acc: 0.7118 - val_precision: 0.4823 - val_recall: 0.4027 - val_f1: 0.4383 - val_TP: 12.7500 - val_FN: 2.7045 - val_TN: 9.8182 - val_FP: 6.4318
Epoch 2/10
12555/12555 [==============================] - 1s 100us/step - loss: 0.1895 - acc: 0.7141 - preci

Train on 12555 samples, validate on 1395 samples
Epoch 1/10
12555/12555 [==============================] - 2s 123us/step - loss: 0.2133 - acc: 0.6589 - precision: 0.6430 - recall: 0.7668 - f1: 0.6805 - TP: 12.2366 - FN: 3.7455 - TN: 8.8142 - FP: 7.1501 - val_loss: 0.1857 - val_acc: 0.7276 - val_precision: 0.4959 - val_recall: 0.4121 - val_f1: 0.4496 - val_TP: 13.0000 - val_FN: 2.7727 - val_TN: 10.0682 - val_FP: 5.8636
Epoch 2/10
12555/12555 [==============================] - 1s 99us/step - loss: 0.1920 - acc: 0.7121 - precision: 0.6856 - recall: 0.7967 - f1: 0.7271 - TP: 12.7226 - FN: 3.2595 - TN: 10.0280 - FP: 5.9364 - val_loss: 0.1807 - val_acc: 0.7355 - val_precision: 0.4959 - val_recall: 0.4092 - val_f1: 0.4477 - val_TP: 12.9091 - val_FN: 2.8636 - val_TN: 10.4091 - val_FP: 5.5227
Epoch 3/10
12555/12555 [==============================] - 1s 98us/step - loss: 0.1851 - acc: 0.7225 - precision: 0.6991 - recall: 0.7927 - f1: 0.7346 - TP: 12.6616 - FN: 3.3206 - TN: 10.4198 - FP: 5.5445 -

12555/12555 [==============================] - 1s 99us/step - loss: 0.1880 - acc: 0.7189 - precision: 0.6924 - recall: 0.7952 - f1: 0.7299 - TP: 12.6387 - FN: 3.2468 - TN: 10.3282 - FP: 5.7328 - val_loss: 0.1814 - val_acc: 0.7341 - val_precision: 0.5227 - val_recall: 0.4155 - val_f1: 0.4622 - val_TP: 13.2273 - val_FN: 3.4091 - val_TN: 10.0455 - val_FP: 5.0227
Epoch 4/10
12555/12555 [==============================] - 1s 99us/step - loss: 0.1834 - acc: 0.7286 - precision: 0.7012 - recall: 0.7958 - f1: 0.7370 - TP: 12.6565 - FN: 3.2290 - TN: 10.6209 - FP: 5.4402 - val_loss: 0.1798 - val_acc: 0.7376 - val_precision: 0.5227 - val_recall: 0.4013 - val_f1: 0.4533 - val_TP: 12.7727 - val_FN: 3.8636 - val_TN: 10.6136 - val_FP: 4.4545
Epoch 5/10
12555/12555 [==============================] - 1s 99us/step - loss: 0.1799 - acc: 0.7352 - precision: 0.7096 - recall: 0.8003 - f1: 0.7441 - TP: 12.6947 - FN: 3.1908 - TN: 10.7913 - FP: 5.2697 - val_loss: 0.1753 - val_acc: 0.7462 - val_precision: 0.5227 